In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
!pip install selenium
!pip install beautifulsoup4

Executing: /tmp/apt-key-gpghome.1o6874PJjh/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Fv73AtXoDo/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.cXxDqHl9Xc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

CalledProcessError: ignored

In [ ]:
# Install chromium and chromium-driver
!apt-get update -y
!apt-get install -y chromium-chromedriver


#dependencies
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install beautifulsoup4

# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException 
import pandas as pd
import os
import sys
import random
import requests
from tqdm import tqdm 
from datetime import datetime
from selenium.webdriver import Chrome
import json

In [ ]:
# 무한 스크롤
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print(" ")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("스크롤 에러 발생: ", e)

In [ ]:
# 데이터 삽입
df1 = pd.read_csv('User_ID.csv') #리뷰어 링크
df2 = pd.read_csv('Review_Category.csv')  #리뷰 카테고리
df1['User_ID']
df2['Review_Category']
Final_df=pd.DataFrame()

#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
from selenium.webdriver.firefox.options import Options
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome('chromedriver', options=options)


for i in range(len(df1)):
  for k in range(len(df2)):

    driver.get("https://m.place.naver.com/my/"+df1['User_ID'][i]+df2['Review_Category'][k])

    #웹페이지 로딩 대기
    time.sleep(3)

    # 무한 스크롤 함수 실행
    scroll()

    # 파싱
    html = driver.page_source 
    soup = BeautifulSoup(html, 'lxml') 
    time.sleep(1)

    # 카테고리별 리뷰 수집
    one_review = soup.find_all('li', attrs = {'class':'_2teBZ'})
    review_num = len(one_review) # 카테고리별 리뷰 개수
    #print(df1['User_ID'][i]+','+df2['카테고리'][k]+','+'리뷰 총 개수'+','+str(review_num))
    
    for r in range(len(one_review)):
      try:
        # 가게명
        store_name = one_review[r].find('span', attrs = {'class':'_2v2rP'}).text
        
        # 가게위치
        shop_location = one_review[r].find('span', attrs = {'class':'_21yZl'}).text
        
        # 텍스트 리뷰
        text = one_review[r].find('div', attrs = {'class':'_1ZYIc'}).text
        
        # 칭찬카테고리
        keyword = one_review[r].find('span', attrs = {'class':'_11i38'}).text

        print(df1['User_ID'][i]+','+df2['카테고리'][k]+','+store_name+','+shop_location+','+text+','+keyword)
      except AttributeError:
        #print(df1['User_ID'][i]+','+df2['카테고리'][k]+','+"none, none")
        store_name='None'
        shop_location='None'
        text='None'
        keyword='None'

        
      Final_Dic={
          'User_ID': [df1['User_ID'][i]],
          'Review_Category': [df2['카테고리'][k]],
          '가게명': [store_name],
          '가게 위치': [shop_location],
          'Review_text': [text],
          'Review_Keyword': [keyword]}
      df_dictionary = pd.DataFrame(Final_Dic)
      Final_df= pd.concat([Final_df, df_dictionary], ignore_index=True)
Final_df.to_csv('final.csv', encoding = 'cp949', index=True)
Final_df

 
5d1d48be0962cc2698984ed3,카페,와플대학 대구월성캠퍼스,대구시 월성동,전체적으로 저렴한 가격대에 맛있어요. 지난번 우연히 찾았다가 맛있길래 재방문 했는데 여전히 맛있네요. 바삭한 도우에 상큼한 아이스크림이 더해진 아이스와플 맛나요 :),매장이 청결해요
5d1d48be0962cc2698984ed3,카페,쁠로13,서울시 논현동,착한 가격에 빵도 맛있어요. 평일 낮 시간대인데도 많이들 찾아오시더니 제대로네요. 그람으로 측정되는 미니 크로와상은 이색적이었어요. 결이 살아있는데다, 저마다 매력 가득해서 다음엔 다른 종류도 먹어볼까 봐요. 크러핀도 진짜 맛있으니 꼭 드셔보세요 :),특별한 메뉴가 있어요
5d1d48be0962cc2698984ed3,카페,하나둘 갤러리카페,서울시 역삼동,갤러리 카페로 볼거리와 먹을거리가 있는 공간이었어요. 가격대도 저렴해서 부담 없는데다, 향긋한 커피 뿐 아니라 디저트도 맛있네요. 갤러리 관람도 잘 하고 왔어요~,디저트가 맛있어요
5d1d48be0962cc2698984ed3,카페,복희빵집,서울시 당산동2가,근처에서 점심 먹으러 왔다가 맛있을거 같아 우연히 찾았어요. 간단하게 포장해왔는데 맛있네요. 선물한 사람도 맛있었다고 해요. 속이 편하고 맛있는 빵. 친절하셔서 더 좋았어요~,특별한 메뉴가 있어요
5d1d48be0962cc2698984ed3,카페,성수동대림창고갤러리,서울시 성수동2가,커피 맛집으로 알고 있었는데 딸기우유도 맛있네요. 인테리어도 특별하고 생각보다 내부도 넓어서 좋았어요. 한가지 아쉬운 점은 딸기우유도 얼음컵을 함께 챙겨주셨으면 더 좋을 거 같아요. 컵 여쭤봤더니 셀프바에 있는 작은 종이컵 사용해야 하더라구요.,매장이 넓어요
5d1d48be0962cc2698984ed3,카페,천장지구,서울시 논현동,반전 매력이 있는 곳이네요. 밖에서 볼 때와는 달리 또 다른 분위기가 근사했어요. 심플한 느낌을 좋아하는데도 불구하고 마음에 쏙 들었던 공간. 하나하나 세심하게 신경 쓰셨다는 것도 느껴지고 메뉴들도 하나같이 맛있

,User_ID,Review_Category,가게명,가게 위치,Review_text,Review_Keyword
0,5d1d48be0962cc2698984ed3,카페,와플대학 대구월성캠퍼스,대구시 월성동,전체적으로 저렴한 가격대에 맛있어요. 지난번 우연히 찾았다가 맛있길래 재방문 했는데...,매장이 청결해요
1,5d1d48be0962cc2698984ed3,카페,쁠로13,서울시 논현동,착한 가격에 빵도 맛있어요. 평일 낮 시간대인데도 많이들 찾아오시더니 제대로네요. ...,특별한 메뉴가 있어요
2,5d1d48be0962cc2698984ed3,카페,하나둘 갤러리카페,서울시 역삼동,갤러리 카페로 볼거리와 먹을거리가 있는 공간이었어요. 가격대도 저렴해서 부담 없는데...,디저트가 맛있어요
3,5d1d48be0962cc2698984ed3,카페,복희빵집,서울시 당산동2가,근처에서 점심 먹으러 왔다가 맛있을거 같아 우연히 찾았어요. 간단하게 포장해왔는데 ...,특별한 메뉴가 있어요
4,5d1d48be0962cc2698984ed3,카페,성수동대림창고갤러리,서울시 성수동2가,커피 맛집으로 알고 있었는데 딸기우유도 맛있네요. 인테리어도 특별하고 생각보다 내부...,매장이 넓어요
...,...,...,...,...,...,...
587,5d1d48be0962cc2698984ed3,기타,None,None,None,None
588,5d1d48be0962cc2698984ed3,기타,None,None,None,None
589,5d1d48be0962cc2698984ed3,기타,None,None,None,None
590,5d1d48be0962cc2698984ed3,기타,None,None,None,None


In [ ]:
# 키워드 추가할 수 있을까?

#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
from selenium.webdriver.firefox.options import Options
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome('chromedriver', options=options)

# 리뷰어 링크
df1 = pd.read_csv('User_ID.csv')
# 리뷰 카테고리
df2 = pd.read_csv('category.csv') 

for i in range(len(df1)):
  for k in range(len(df2)):

    driver.get("https://m.place.naver.com/my/"+df1['User_ID'][i]+df2['category'][k])

    #웹페이지 로딩 대기
    time.sleep(3)

    # 무한 스크롤 함수 실행
    scroll()


     # 파싱
    html = driver.page_source 
    soup = BeautifulSoup(html, 'lxml') 
    time.sleep(1)

    
    # 카테고리별 리뷰 수집
    one_review = soup.find_all('li', attrs = {'class':'_2teBZ'})
    review_num = len(one_review) # 카테고리별 리뷰 개수
    #print(df1['User_ID'][i]+','+df2['카테고리'][k]+','+'리뷰 총 개수'+','+str(review_num))
    
    for r in range(len(one_review)):
      try:
        # 가게명
        store_name = one_review[r].find('span', attrs = {'class':'_2v2rP'}).text
        
        # 가게위치
        shop_location = one_review[r].find('span', attrs = {'class':'_21yZl'}).text
        
        # 텍스트 리뷰
        text = one_review[r].find('div', attrs = {'class':'_1ZYIc'}).text
        
        # 칭찬카테고리
        keyword = one_review[r].find('span', attrs = {'class':'_11i38'}).text

        print(df1['User_ID'][i]+','+df2['카테고리'][k]+','+store_name+','+shop_location+','+text+','+keyword)
      except AttributeError:
        #print(df1['User_ID'][i]+','+df2['카테고리'][k]+','+"none, none")
        store_name='None'
        shop_locatione='None'
        texte='None'
        keyworde='None'

        
      Final_Dic={
          'User_ID': [df1['User_ID'][i]],
          'Review_Category': [df2['카테고리'][k]],
          '가게명': [store_name],
          '가게 위치': [shop_location],
          'Review_text': [text],
          'Review_Keyword': [keyword]}
      df_dictionary = pd.DataFrame(Final_Dic)
      Final_df= pd.concat([Final_df, df_dictionary], ignore_index=True)
Final_df.to_csv('final.csv', encoding = 'utf8', index=True)
Final_df